<a href="https://colab.research.google.com/github/brayanvelasquez/pavement-1/blob/brayanvelasquez-patch-1/Model_pavement_strains1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import scipy.optimize as spopt
import os
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras import layers
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
with open("/gdrive/MyDrive/A.Pitra/01.simulaciones.txt", 'r') as file:
    lines = file.read()


In [ ]:

nombre_colum = ['dx', 'dy', 'dz', 'DEF(xx)', 'DEF(zz)', 'DEF(xy)',
                'DEF(xz)', 'DEF(yz)','Sxx','Syy','Szz','Sxy','Sxz', 'Syz', 'Y', 'Z',
                'Eca', 'Hca', 'Eb', 'Hb','Esb', 'Hsb']

data2=pd.read_csv("/gdrive/MyDrive/A.Pitra/01.simulaciones.txt",names=nombre_colum, header=None)

data2.head()

,dx,dy,dz,DEF(xx),DEF(zz),DEF(xy),DEF(xz),DEF(yz),Sxx,Syy,...,Sxz,Syz,Y,Z,Eca,Hca,Eb,Hb,Esb,Hsb
0,-9.470700e-22,-0.000015,0.000421,0.000253,-0.000054,-9.887100e-21,1.521200e-36,2.484300e-20,1329800.0,1128000.0,...,1.901500e-27,3.105300e-11,0.162,0.00,3.250000e+09,0.1,250000000.0,0.25,200000000.0,0.25
1,-6.192600e-22,0.000010,0.000412,-0.000228,0.000205,9.094100e-21,8.871000e-22,1.448700e-05,-901830.0,-716180.0,...,1.108900e-12,1.810900e+04,0.162,0.10,3.250000e+09,0.1,250000000.0,0.25,200000000.0,0.25
2,-7.863700e-22,0.000013,0.000383,-0.000198,0.000512,9.214100e-21,1.276000e-20,2.083900e-04,4433.6,18367.0,...,1.181500e-12,1.929500e+04,0.162,0.15,3.250000e+09,0.1,250000000.0,0.25,200000000.0,0.25
3,-8.687500e-22,0.000014,0.000361,-0.000171,0.000407,8.283600e-21,1.251300e-20,2.043500e-04,-2781.3,9744.8,...,1.158600e-12,1.892100e+04,0.162,0.20,3.250000e+09,0.1,250000000.0,0.25,200000000.0,0.25
4,-9.078800e-22,0.000015,0.000342,-0.000150,0.000334,7.082800e-21,1.157500e-20,1.890300e-04,-8047.6,2662.7,...,1.071700e-12,1.750300e+04,0.162,0.25,3.250000e+09,0.1,250000000.0,0.25,200000000.0,0.25


In [ ]:
####################################
####################################
######eliminar cierta columna del archivo


# Importar la librería csv
import csv

# Definir el nombre del archivo de entrada y salida
nombre_archivo = "/gdrive/MyDrive/A.Pitra/0.simulaciones.txt"
nombre_archivo_actualizado = "/gdrive/MyDrive/A.Pitra/01.simulaciones.txt"

# Leer el archivo de entrada y almacenar los datos en una lista
datos = []
with open(nombre_archivo, 'r') as archivo_entrada:
    lector_csv = csv.reader(archivo_entrada)
    for fila in lector_csv:
        datos.append(fila)

# Eliminar la segunda columna de la lista
for fila in datos:
    del fila[4]

# Guardar la lista actualizada en el mismo archivo o en uno nuevo
with open(nombre_archivo_actualizado, 'w', newline='') as archivo_salida:
    escritor_csv = csv.writer(archivo_salida)
    for fila in datos:
        escritor_csv.writerow(fila)

In [ ]:
data2.describe()


,dx,dy,dz,DEF(xx),DEF(zz),DEF(xy),DEF(xz),DEF(yz),Sxx,Syy,...,Sxz,Syz,Y,Z,Eca,Hca,Eb,Hb,Esb,Hsb
count,6.017000e+03,6.017000e+03,6017.000000,6017.000000,6017.000000,6.017000e+03,6.017000e+03,6.017000e+03,6.017000e+03,6.017000e+03,...,6.017000e+03,6.017000e+03,6017.000000,6017.000000,6.017000e+03,6017.000000,6.017000e+03,6017.000000,6.017000e+03,6017.000000
mean,-1.888451e-21,-6.922011e-08,0.000321,-0.000139,0.000346,1.007076e-20,-7.872424e-21,3.545357e-07,3.063342e+04,3.170369e+04,...,-4.070017e-13,3.736665e+01,0.005035,1.035923,2.422428e+09,0.061745,2.002327e+08,0.173533,1.521522e+08,0.173533
std,1.759922e-21,1.892709e-05,0.000197,0.000199,0.000369,1.814051e-20,2.286337e-20,1.766283e-04,4.124580e+05,3.609344e+05,...,1.270651e-12,9.765430e+03,0.161935,1.126512,7.106804e+08,0.025521,4.049370e+07,0.055383,3.983140e+07,0.055383
min,-8.800500e-21,-4.790700e-05,0.000055,-0.000885,-0.000370,-6.079000e-20,-9.234100e-20,-5.026800e-04,-1.856900e+06,-1.689400e+06,...,-4.585300e-12,-2.496100e+04,-0.162000,0.000000,1.500000e+09,0.030000,1.500000e+08,0.100000,1.000000e+08,0.100000
25%,-2.805400e-21,-1.550500e-05,0.000199,-0.000228,0.000079,2.527000e-22,-1.790500e-20,-9.747000e-05,-4.881000e+03,-5.670900e+02,...,-6.564000e-13,-3.573300e+03,-0.162000,0.300000,1.500000e+09,0.030000,1.500000e+08,0.100000,1.000000e+08,0.100000
50%,-1.284500e-21,7.930100e-07,0.000286,-0.000115,0.000262,4.401200e-21,3.942000e-36,4.890300e-20,-1.405100e+03,-2.088200e+02,...,3.107800e-27,4.513000e-11,0.162000,0.650000,2.500000e+09,0.050000,2.000000e+08,0.150000,1.500000e+08,0.150000
75%,-5.851000e-22,1.521100e-05,0.000435,-0.000030,0.000465,1.417200e-20,6.023100e-21,9.836400e-05,-7.839800e+01,1.104400e+02,...,2.243300e-13,3.663600e+03,0.162000,1.000000,3.250000e+09,0.070000,2.500000e+08,0.200000,2.000000e+08,0.200000
max,-4.169800e-24,4.790700e-05,0.000989,0.000736,0.002561,1.035100e-19,3.078000e-20,5.026800e-04,3.077900e+06,2.783400e+06,...,1.528400e-12,2.496100e+04,0.162000,4.000000,3.250000e+09,0.100000,2.500000e+08,0.250000,2.000000e+08,0.250000


In [ ]:
data2[1]

In [ ]:
fig, axes = plt.subplots(figsize=(18,5),nrows=1,ncols=3)

axes[0].scatter(data2[15],data2[0],color="b")
axes[0].set_xlabel("variables X")
axes[0].set_ylabel("Variables Y")
axes[0].set_title("")

axes[1].scatter(data2[16],data2[0],color="r")
axes[1].set_xlabel("variables X")
axes[1].set_ylabel("Variables Y")
axes[1].set_title("")

axes[2].scatter(data2[17],data2[0],color="r")
axes[2].set_xlabel("variables X")
axes[2].set_ylabel("Variables Y")
axes[2].set_title("")


In [ ]:
sns.pairplot(data2[['Sxx','Syy','Szz','Sxy','Sxz', 'Syz']], diag_kind="kde")

In [ ]:
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################
################################################################################

In [ ]:
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA

# cargar los datos
data = np.loadtxt("/gdrive/MyDrive/A.Pitra/01.simulaciones.txt", delimiter=",")

# dividir los datos en características (X) y etiquetas (y)
X = data[:,14:22]
y = data[:,2:5]

##############################################################################
##############################################################################
##############################################################################
# Normalizar los datos
#means = np.mean(X)
#stds = np.std(X)
meany = np.mean(y, axis=0)
stdy = np.std(y, axis=0)
# Normalizar cada columna de la matriz X
y_norm = (y - meany) / stdy
##############################################################################
##############################################################################
##############################################################################

mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
# Normalizar cada columna de la matriz X
X_norm = (X - mean) / std

X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2)


#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# definir el modelo
def crear_modelo():
 modelo = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_dim=8, activation='tanh', kernel_initializer="glorot_uniform"),
  tf.keras.layers.Dense(256, activation='tanh', kernel_initializer="glorot_uniform"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(512, activation='tanh', kernel_initializer="glorot_uniform"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu', kernel_initializer="he_normal"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='tanh', kernel_initializer="glorot_uniform"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu', kernel_initializer="he_normal"),
  tf.keras.layers.Dense(3, activation='linear')
 ])
 opt = keras.optimizers.SGD(learning_rate=0.07)
 modelo.compile(loss='mean_squared_error', optimizer=opt, metrics=['mse'])
 return modelo


# definir el número de folds
num_folds = 5

# inicializar el objeto KFold
kfold = KFold(n_splits=num_folds, shuffle=True)

# realizar la validación cruzada
scores = []
y_test_preds = []
y_val_preds = []
test_mses = []
test_maes = []
test_r2s = []
val_r2s = []
entre_r2s = []
entre_mses = []
test_r2svariable1= []
test_r2svariable2=[]
test_r2svariable3=[]
entre_r2svariable1 = []
entre_r2svariable2 = []
entre_r2svariable3 = []
entre_msesv1 =[]
entre_msesv2 =[]
entre_msesv3 =[]
test_msesv1 =[]
test_msesv2 =[]
test_msesv3 =[]

for train, test in kfold.split(X_train):
  # crear el modelo
  modelo = crear_modelo()
  early = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=0, start_from_epoch=0)
  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=6, verbose=1, min_lr=0.00006)

  # entrenar el modelo
  modelo.fit(X_train[train], y_train[train], epochs=3000, validation_split=0.2, batch_size=10, callbacks=[reduce_lr, early])
  #modelo.fit(X_train[train], y_train[train], epochs=500, validation_split=0.2, batch_size=10)

  # evaluar el modelo
  score = modelo.evaluate(X_train[test], y_train[test], verbose=0)
  scores.append(score[1])

  ##############################################################################
  ##############################################################################
  ##############################################################################

  y_test_pred = modelo.predict(X_train[test])
  #y_test_preds.append(y_test_pred[1])
  y_val_pred = modelo.predict(X_train[test])
 # y_val_preds.append(y_val_pred[1])
  y_entre_pred = modelo.predict(X_train[train])
 # y_val_preds.append(y_val_pred[1])
  y1=y_train[train]
  y2=y_train[test]
  
  ##### Tres Variables #####

  entre_mse = mean_squared_error(y1, y_entre_pred)
  entre_mses.append(entre_mse)

  test_mse = mean_squared_error(y2, y_test_pred)
  test_mses.append(test_mse)

  test_r2 = r2_score(y2, y_test_pred)
  test_r2s.append(test_r2)

  entre_r2 = r2_score(y1, y_entre_pred)
  entre_r2s.append(entre_r2)

  val_r2 = r2_score(y2, y_val_pred)
  val_r2s.append(val_r2)

  ##### Variables 1 #####

  entre_msev1 = mean_squared_error(y1[0], y_entre_pred[0])
  entre_msesv1.append(entre_msev1)

  test_msev1 = mean_squared_error(y2[0], y_test_pred[0])
  test_msesv1.append(test_msev1)

  test_r2variable1 = r2_score(y2[0], y_test_pred[0])
  test_r2svariable1.append(test_r2variable1)

  entre_r2variable1 = r2_score(y1[0], y_entre_pred[0])
  entre_r2svariable1.append(entre_r2variable1)

  ##### Variables 2 ####

  entre_msev2 = mean_squared_error(y1[1], y_entre_pred[1])
  entre_msesv2.append(entre_msev2)

  test_msev2 = mean_squared_error(y2[1], y_test_pred[1])
  test_msesv2.append(test_msev2)

  test_r2variable2 = r2_score(y2[1], y_test_pred[1])
  test_r2svariable2.append(test_r2variable2)

  entre_r2variable2 = r2_score(y1[1], y_entre_pred[1])
  entre_r2svariable2.append(entre_r2variable2)

  ##### Variables 3 #####

  entre_msev3 = mean_squared_error(y1[2], y_entre_pred[2])
  entre_msesv3.append(entre_msev3)

  test_msev3 = mean_squared_error(y2[2], y_test_pred[2])
  test_msesv3.append(test_msev3)

  test_r2variable3 = r2_score(y2[2], y_test_pred[2])
  test_r2svariable3.append(test_r2variable3)

  entre_r2variable3 = r2_score(y1[2], y_entre_pred[2])
  entre_r2svariable3.append(entre_r2variable3)

###############################################################################
###############################################################################

  print("Variable1/////////R^2 Test:", test_r2variable1, "///////R^2 Entre:", entre_r2variable1)
  print("Variable2/////////R^2 Test:", test_r2variable2, "///////R^2 Entre:", entre_r2variable2)
  print("Variable3/////////R^2 Test:", test_r2variable3, "///////R^2 Entre:", entre_r2variable3)
  print("##################################")
  print("Tres Variables/////////R^2 Test:", test_r2, "///////R^2 Entre:", entre_r2, "///////R^2 Val:", val_r2)
###############################################################################
###############################################################################
 
# calcular la media y la desviación estándar de los resultados
print("##################################")


print("Métricas:")
print("Tres Variables/////////MSE Test:", np.mean(test_mses), "+/-", np.std(test_mses), "///////MSE Entre:", np.mean(entre_mses), "+/-", np.std(entre_mses), "///////MSE Val:", np.mean(scores), "+/-", np.std(scores))
print("Variables # 1/////////MSE Test:", np.mean(test_msesv1), "+/-", np.std(test_msesv1), "///////MSE Entre:", np.mean(entre_msesv1), "+/-", np.std(entre_msev1))
print("Variables # 2/////////MSE Test:", np.mean(test_msesv2), "+/-", np.std(test_msesv2), "///////MSE Entre:", np.mean(entre_msesv2), "+/-", np.std(entre_msev2))
print("Variables # 3/////////MSE Test:", np.mean(test_msesv3), "+/-", np.std(test_msesv3), "///////MSE Entre:", np.mean(entre_msesv3), "+/-", np.std(entre_msev3))

print("##################################")
print("Tres Variables/////////R^2 Test:", np.mean(test_r2s), "+/-", np.std(test_r2s), "///////R^2 Entre:", np.mean(entre_r2s), "+/-", np.std(entre_r2s), "///////R^2 Val:", np.mean(val_r2s), "+/-", np.std(val_r2s))
print("Variables # 1/////////R^2 Test:", np.mean(test_r2svariable1), "+/-", np.std(test_r2svariable1), "///////R^2 Entre:", np.mean(entre_r2svariable1), "+/-", np.std(entre_r2svariable1))
print("Variables # 2/////////R^2 Test:", np.mean(test_r2svariable2), "+/-", np.std(test_r2svariable2), "///////R^2 Entre:", np.mean(entre_r2svariable2), "+/-", np.std(entre_r2svariable2))
print("Variables # 3/////////R^2 Test:", np.mean(test_r2svariable3), "+/-", np.std(test_r2svariable3), "///////R^2 Entre:", np.mean(entre_r2svariable3), "+/-", np.std(entre_r2svariable3))



Epoch 1/3000
308/308 [==============================] - 2s 5ms/step - loss: 0.5634 - mse: 0.5634 - val_loss: 0.3928 - val_mse: 0.3928 - lr: 0.0700
Epoch 2/3000
308/308 [==============================] - 2s 5ms/step - loss: 0.4338 - mse: 0.4338 - val_loss: 0.3127 - val_mse: 0.3127 - lr: 0.0700
Epoch 3/3000
308/308 [==============================] - 2s 6ms/step - loss: 0.4059 - mse: 0.4059 - val_loss: 0.3497 - val_mse: 0.3497 - lr: 0.0700
Epoch 4/3000
308/308 [==============================] - 1s 5ms/step - loss: 0.3918 - mse: 0.3918 - val_loss: 0.3970 - val_mse: 0.3970 - lr: 0.0700
Epoch 5/3000
308/308 [==============================] - 1s 4ms/step - loss: 0.3791 - mse: 0.3791 - val_loss: 0.2836 - val_mse: 0.2836 - lr: 0.0700
Epoch 6/3000
308/308 [==============================] - 1s 4ms/step - loss: 0.3926 - mse: 0.3926 - val_loss: 0.2956 - val_mse: 0.2956 - lr: 0.0700
Epoch 7/3000
308/308 [==============================] - 1s 4ms/step - loss: 0.3701 - mse: 0.3701 - val_loss: 0.3356 - 

In [ ]:
modelo.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 256)               2304      
                                                                 
 dense_29 (Dense)            (None, 256)               65792     
                                                                 
 dropout_16 (Dropout)        (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 512)               131584    
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                                 
 dense_31 (Dense)            (None, 256)               131328    
                                                                 
 dropout_18 (Dropout)        (None, 256)              

In [ ]:
print("Accuracy: ", (np.square(np.mean(scores)), np.std(scores)))


346851628.8998518
[[ 8.22991274e-13  0.00000000e+00  1.65106274e-02 ...  1.27004826e-12
   1.01603861e-03  1.27004826e-12]
 [ 8.22991274e-13  5.08019305e-13  1.65106274e-02 ...  1.27004826e-12
   1.01603861e-03  1.27004826e-12]
 [ 8.22991274e-13  7.62028958e-13  1.65106274e-02 ...  1.27004826e-12
   1.01603861e-03  1.27004826e-12]
 ...
 [-8.22991274e-13  1.52405792e-11  1.27004826e-02 ...  5.08019305e-13
   5.08019305e-04  5.08019305e-13]
 [-8.22991274e-13  1.77806757e-11  1.27004826e-02 ...  5.08019305e-13
   5.08019305e-04  5.08019305e-13]
 [-8.22991274e-13  2.03207722e-11  1.27004826e-02 ...  5.08019305e-13
   5.08019305e-04  5.08019305e-13]]
[[ 1.62e-01  0.00e+00  3.25e+09 ...  2.50e-01  2.00e+08  2.50e-01]
 [ 1.62e-01  1.00e-01  3.25e+09 ...  2.50e-01  2.00e+08  2.50e-01]
 [ 1.62e-01  1.50e-01  3.25e+09 ...  2.50e-01  2.00e+08  2.50e-01]
 ...
 [-1.62e-01  3.00e+00  2.50e+09 ...  1.00e-01  1.00e+08  1.00e-01]
 [-1.62e-01  3.50e+00  2.50e+09 ...  1.00e-01  1.00e+08  1.00e-01]
 [-1.6

In [ ]:
X = data[:,15:23]
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)

# Normalizar cada columna de la matriz X
X_normalized = (X - mean) / std

# Imprimir la matriz normalizada
print(X_normalized)

[[ 0.96938966 -0.9196614   1.16457505 ...  1.38080542  1.20135717
   1.38080542]
 [ 0.96938966 -0.83088441  1.16457505 ...  1.38080542  1.20135717
   1.38080542]
 [ 0.96938966 -0.78649592  1.16457505 ...  1.38080542  1.20135717
   1.38080542]
 ...
 [-1.03157692  1.74364815  0.10916064 ... -1.32783603 -1.30943341
  -1.32783603]
 [-1.03157692  2.18753308  0.10916064 ... -1.32783603 -1.30943341
  -1.32783603]
 [-1.03157692  2.631418    0.10916064 ... -1.32783603 -1.30943341
  -1.32783603]]


In [ ]:
X1= np.array([(X[1]-mean)/std])
y_pred = modelo.predict(X1)
yreal = (y[1]-meansy)/stdsy
Error=100*np.abs(y_pred-yreal)/np.abs(yreal)
print("y_pred:",y_pred)
print("y_real:",yreal)
print(Error)

1/1 [==============================] - 0s 21ms/step
y_pred: [[0.32492766]]
y_real: 0.4628366797366907
[[29.796476]]


In [ ]:
X1= np.array([(X[5125]-mean)/std])
y_pred1 = modelo.predict(X1)
y_pred = (y_pred1*stdsy)+meansy
yreal = y[5125]
Error=100*np.abs(yreal-y_pred)/np.abs(yreal)
print("y_pred:",y_pred)
print("y_real:",yreal)
print(Error)

1/1 [==============================] - 0s 37ms/step
y_pred: [[0.00025003]]
y_real: 0.00027923
[[10.456595]]


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
a1=[1708.02,1115.81,721.07,457.82,240.02,121.36,99.04,54.92,33.61]
b1=[1706,1112,723.10,458.10,245.10,123.9,100.10,55.42,34.42]
mse = mean_squared_error(b1,a1)
print(mse)
print(np.sqrt(mse))
print(np.std(mse))

6.342611111111029
2.518454111376864
0.0
